<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block; font-weight:bold; font-size:40px;">
Notebook 4: SQL Server Loading
</p>

<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block;">
1. Database Setup<br>
- Create RetailDB database<br>
- Create tables: Brands, Categories, Products, Customers, Orders, OrderItems, Staffs, Stores, Stocks<br>
- Define primary and foreign keys
</p>

<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block;">
2. Load Data<br>
- Upload cleaned data from Pandas to SQL Server using SQLAlchemy
</p>


_______

<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block;
font-weight:bold; font-size:30px;">
Server Connection
</p>


In [1]:
from sqlalchemy import create_engine

server = r'DESKTOP-05DRQII'
database = 'RetailDB'
driver = "ODBC Driver 17 for SQL Server"  # أو 18 حسب ما عندك

engine = create_engine(
    f"mssql+pyodbc://@{server}/?driver={driver}&trusted_connection=yes"
)


In [2]:
engine = create_engine(
    f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes"
)


<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block;
font-weight:bold; font-size:30px;">
Create tables
</p>


In [3]:
from sqlalchemy import create_engine, text

#---------------------------------------
server = r'DESKTOP-05DRQII'   
database = 'RetailDB'
driver = "ODBC Driver 17 for SQL Server"  

engine = create_engine(
    f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes"
)

#---------------------------------------
with engine.connect() as conn:

    # 1. Brands
    conn.execute(text("""
    IF OBJECT_ID('Brands', 'U') IS NULL
    CREATE TABLE Brands (
        brand_id INT PRIMARY KEY,
        brand_name NVARCHAR(100)
    )
    """))

    # 2. Categories
    conn.execute(text("""
    IF OBJECT_ID('Categories', 'U') IS NULL
    CREATE TABLE Categories (
        category_id INT PRIMARY KEY,
        category_name NVARCHAR(100)
    )
    """))

    # 3. Customers
    conn.execute(text("""
    IF OBJECT_ID('Customers', 'U') IS NULL
    CREATE TABLE Customers (
        customer_id INT PRIMARY KEY,
        first_name NVARCHAR(50),
        last_name NVARCHAR(50),
        full_name NVARCHAR(100),
        phone NVARCHAR(50),
        email NVARCHAR(100),
        street NVARCHAR(100),
        city NVARCHAR(50),
        state NVARCHAR(50),
        zip_code NVARCHAR(20)
    )
    """))

    # 4. Stores
    conn.execute(text("""
    IF OBJECT_ID('Stores', 'U') IS NULL
    CREATE TABLE Stores (
        store_id INT PRIMARY KEY,
        store_name NVARCHAR(100),
        phone NVARCHAR(50),
        email NVARCHAR(100),
        street NVARCHAR(100),
        city NVARCHAR(50),
        state NVARCHAR(50),
        zip_code NVARCHAR(20)
    )
    """))

    # 5. Products (FK → Brands, Categories)
    conn.execute(text("""
    IF OBJECT_ID('Products', 'U') IS NULL
    CREATE TABLE Products (
        product_id INT PRIMARY KEY,
        product_name NVARCHAR(100),
        brand_id INT,
        category_id INT,
        model_year INT,
        list_price FLOAT,
        FOREIGN KEY (brand_id) REFERENCES Brands(brand_id),
        FOREIGN KEY (category_id) REFERENCES Categories(category_id)
    )
    """))

    # 6. Staffs (FK → Stores)
    conn.execute(text("""
    IF OBJECT_ID('Staffs', 'U') IS NULL
    CREATE TABLE Staffs (
        staff_id INT PRIMARY KEY,
        first_name NVARCHAR(50),
        last_name NVARCHAR(50),
        email NVARCHAR(100),
        phone NVARCHAR(50),
        store_id INT,
        active BIT,
        username NVARCHAR(50),
        FOREIGN KEY (store_id) REFERENCES Stores(store_id)
    )
    """))

    # 7. Orders (FK → Customers, Stores, Staffs)
    conn.execute(text("""
    IF OBJECT_ID('Orders', 'U') IS NULL
    CREATE TABLE Orders (
        order_id INT PRIMARY KEY,
        customer_id INT,
        order_status NVARCHAR(50),
        order_date DATETIME,
        required_date DATETIME,
        shipped_date DATETIME,
        store_id INT,
        staff_id INT,
        order_total FLOAT,
        FOREIGN KEY (customer_id) REFERENCES Customers(customer_id),
        FOREIGN KEY (store_id) REFERENCES Stores(store_id),
        FOREIGN KEY (staff_id) REFERENCES Staffs(staff_id)
    )
    """))

    # 8. OrderItems (FK → Orders, Products)
    conn.execute(text("""
    IF OBJECT_ID('OrderItems', 'U') IS NULL
    CREATE TABLE OrderItems (
        order_item_id INT PRIMARY KEY,
        order_id INT,
        product_id INT,
        quantity INT,
        list_price FLOAT,
        discount FLOAT,
        total_price FLOAT,
        FOREIGN KEY (order_id) REFERENCES Orders(order_id),
        FOREIGN KEY (product_id) REFERENCES Products(product_id)
    )
    """))

    # 9. Stocks (FK → Stores, Products)
    conn.execute(text("""
    IF OBJECT_ID('Stocks', 'U') IS NULL
    CREATE TABLE Stocks (
        store_id INT,
        product_id INT,
        quantity INT,
        PRIMARY KEY(store_id, product_id),
        FOREIGN KEY(store_id) REFERENCES Stores(store_id),
        FOREIGN KEY(product_id) REFERENCES Products(product_id)
    )
    """))

print("All tables created successfully ✅")


All tables created successfully ✅


_________

<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block;
font-weight:bold; font-size:30px;">
data upload
</p>


In [4]:
import pandas as pd

brands       = pd.read_csv("brands.csv")
categories   = pd.read_csv("categories.csv")
products     = pd.read_csv("products.csv")
customers    = pd.read_csv("customers.csv")
stores       = pd.read_csv("stores.csv")
staffs       = pd.read_csv("staffs.csv")
orders       = pd.read_csv("orders.csv")
order_items  = pd.read_csv("order_items.csv")
stocks       = pd.read_csv("stocks.csv")


In [5]:
# إنشاء full_name للعملاء
customers["full_name"] = customers["first_name"] + " " + customers["last_name"]

# دمج Products مع Brands و Categories
products_merged = products.merge(brands, on="brand_id", how="left") \
                            .merge(categories, on="category_id", how="left")

# حساب total_price في order_items
order_items["total_price"] = order_items["quantity"] * order_items["list_price"]


In [6]:
brands.to_sql("Brands", engine, if_exists="append", index=False)
categories.to_sql("Categories", engine, if_exists="append", index=False)
customers.to_sql("Customers", engine, if_exists="append", index=False)
stores.to_sql("Stores", engine, if_exists="append", index=False)

products_merged.to_sql("Products", engine, if_exists="append", index=False)
staffs.to_sql("Staffs", engine, if_exists="append", index=False)
orders.to_sql("Orders", engine, if_exists="append", index=False)
order_items.to_sql("OrderItems", engine, if_exists="append", index=False)
stocks.to_sql("Stocks", engine, if_exists="append", index=False)


240

In [7]:
from sqlalchemy import create_engine
import pandas as pd

# ---------- إعداد الاتصال ----------
server = r'DESKTOP-05DRQII'
database = 'RetailDB'
driver = "ODBC Driver 17 for SQL Server"

engine = create_engine(
    f"mssql+pyodbc://@{server}/{database}?driver={driver}&trusted_connection=yes"
)

# ---------- رفع البيانات ----------
# تأكد أن هذه DataFrames موجودة بعد تنظيفك وتحويلك
# مثال:
# products_merged, brands, categories, customers, stores, staffs, orders, order_items, stocks

# 1️⃣ الجداول الأب
brands.to_sql("Brands", engine, if_exists="append", index=False)
categories.to_sql("Categories", engine, if_exists="append", index=False)
customers.to_sql("Customers", engine, if_exists="append", index=False)
stores.to_sql("Stores", engine, if_exists="append", index=False)

# 2️⃣ الجداول التي تحتوي FK
products_merged.to_sql("Products", engine, if_exists="append", index=False)
staffs.to_sql("Staffs", engine, if_exists="append", index=False)
orders.to_sql("Orders", engine, if_exists="append", index=False)
order_items.to_sql("OrderItems", engine, if_exists="append", index=False)
stocks.to_sql("Stocks", engine, if_exists="append", index=False)

print("All data uploaded successfully ✅")


All data uploaded successfully ✅


In [8]:
import pandas as pd

tables = ["Brands", "Categories", "Products", "Customers", "Stores",
            "Staffs", "Orders", "OrderItems", "Stocks"]

for table in tables:
    df = pd.read_sql(f"SELECT COUNT(*) AS total_rows FROM {table}", engine)
    print(f"{table}: {df['total_rows'][0]} rows")


Brands: 36 rows
Categories: 28 rows
Products: 1336 rows
Customers: 5780 rows
Stores: 12 rows
Staffs: 40 rows
Orders: 6460 rows
OrderItems: 19056 rows
Stocks: 3756 rows


______________________

<p style="background: linear-gradient(to right, #1E90FF, #8A2BE2); -webkit-background-clip: text; color: transparent; display:block;
font-weight:bold; font-size:30px;">
Finish 🎉
</p>
